In [ ]:
#change path, the directory where the source files are located
import os
os.chdir("..")

In [ ]:
import pandas as pd
load_model = True
model_dir = 'input/ber-sklearn-mlp/'
clf_model = 'mlp'
data_type = 'processed'
data_dir = f'input/ber-processed-parquet/'


# Read the processed data from the parquet file
processed = pd.read_parquet(f"{data_dir}/processed.parquet")

# Separate the features (X_processed) and labels (labels)
X_processed = processed.drop(columns=["EnergyRating"])
labels = processed['EnergyRating']

In [ ]:
import numpy as np
from clear.utils import load_train_test, load_embedding
n_splits = 5
pred_probs = []
embedding_dir = 'input/ber-scarf-train-model/'
data_dir = f'input/ber-processed-parquet/'


model_dir = 'input/ber-sklearn-models/processed/mlp'
clf_model = 'mlp'
data_type = 'processed'



In [ ]:
from clear.utils import load_pickle
from clear.model_sklearn import get_pred_probs
pred_probs_proc = []
labels_proc = []
for i in range(n_splits):
    print(f"split {i+1}")
    X_train, y_train, X_test, y_test = load_train_test(f'{data_dir}/split_{i+1}', data_type = 'processed', data_format='parquet')
    model = load_pickle(f'{model_dir}/split_{i+1}/{clf_model}_{data_type}.pkl')
    pred = get_pred_probs(model, X_test)
    pred_probs_proc.append(pred)
    labels_proc.append(y_test)

In [ ]:
from clear.confident_learning import find_merged_classes
from clear.utils import get_ber_class_mapping

class_names, class_mapping = get_ber_class_mapping()
overlapped, merged, merged_dict = find_merged_classes(
    labels_proc[0], class_names, pred_probs_proc[0], thresh=0.02
)
merged_dict

In [ ]:
df_merged = pd.DataFrame.from_dict(merged_dict, orient='index')
df_merged = df_merged.reset_index()
df_merged.columns = ['origin', 'merged']
output_dir = 'output/'
df_merged.to_csv(f'{output_dir}/merged.csv', index=False)

In [ ]:
from clear.confident_learning import merge_label
from clear.utils import get_scores, save_model_pickle
from clear.model_sklearn import get_clf_mlp, get_clf_rf

compute_processed = True
model_type = 'rf'

for i in range(n_splits):
    X_train, y_train, X_test, y_test = load_train_test(f'{data_dir}/split_{i+1}', data_type = 'processed', data_format='parquet')
    y_train_merged = merge_label(y_train, merged_dict)
    y_test_merged = merge_label(y_test, merged_dict)

    #processed
    if compute_processed:
        #use the following to load a pretrained model
        # model = load_pickle(f'{model_dir}/split_{i+1}/{clf_model}_{data_type}.pkl')
        #use sklearn model from scratch (mlp pytorch model is implemented under the pytorch folder)

        if model_type == 'mlp':
            clf, clf_model = get_clf_mlp()
        elif model_type == 'rf':
            clf, clf_model = get_clf_rf()
        else:
            raise('Model type not supported')

        clf.fit(X_train, y_train_merged)
        save_model_pickle(output_dir, clf, 'processed', clf_model)
        y_pred = clf.predict(X_test)
        scores = get_scores(y_test_merged, y_pred)
        scores_emb = pd.DataFrame.from_dict(scores)
        scores_emb.to_csv(f'{output_dir}/scores_processed_{i+1}.csv', index=False)


In [ ]:
overlapped.head(10)

In [ ]:
import pandas as pd

def get_scores_mean(folder, fname, n_splits=5):
    scores = []
    for i in range(n_splits):
        curr = pd.read_csv(f'{folder}/{fname}_{i+1}.csv')
        scores.append(curr)

    scores = pd.concat(scores)
    return scores.mean(axis=0)

avg_scores = get_scores_mean(output_dir, 'scores_processed')
avg_scores.to_csv(f'{output_dir}/avg_scores.csv', index=False)
avg_scores